In [41]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

In [9]:
# !pip install torch -i https://pypi.tuna.tsinghua.edu.cn/simple

In [42]:
# Create Tensors to hold input and outputs.

#第一组参数
# beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
# beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
# beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

#第二组参数

beta_1=np.append(np.array([4,4,-8,-4,6,-4,8,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,2,-4,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,3,-6,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0
prior=0

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    
def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [43]:
################## first step##################   
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)


    sum_tGM=0
    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lambda2,lr= config["lambda1"], config["lambda2"],config["lr"]
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1[idx_train])
            out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2[idx_train])
            out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params = model_1.linear1
            model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
            model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
            model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])
           

            model_2_all_linear1_params = model_1.linear1
            model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
            model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
            model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])
    

            model_3_all_linear1_params = model_3.linear1
            model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
            model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
            model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                             model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                             model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
#         test_out_1, test_layer1_out_1, test_layer2_out_1,test_layer3_out_1= model_1(train_x1[idx_test])
#         test_out_2, test_layer1_out_2, test_layer2_out_2,test_layer3_out_2= model_2(train_x2[idx_test])
#         test_out_3, test_layer1_out_3, test_layer2_out_3,test_layer3_out_3= model_3(train_x3[idx_test])
        
#         test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
#         test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
#         test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
#         test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
#         sum_test_BCE_loss+=test_BCE_loss.item()

#     #print("loss: ",)
#     tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

        test_prediction_1= (model_1(train_x1[idx_test])[0]>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])[0]>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])[0]>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)


In [44]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [45]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)
    
#     print(sigmoid(x1@beta_1).squeeze().cov())

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
    
################## first step##################   
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out
    
    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
        "lambda2": tune.choice([0.001,0.002,0.003]),
        "lr":tune.choice([0.6,0.7,0.8])
    }

    result = tune.run(
        train_model,  
        config=config,
        num_samples=30)
    
################## second step##################   

    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,lambda2,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]

    model_1=MLP(seed=1)
    model_2=MLP(seed=1)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

# learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
# loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1)
        out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2)
        out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_1_all_linear1_params = model_1.linear1
        model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
        model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
        model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])

        model_2_all_linear1_params = model_1.linear1
        model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
        model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
        model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])

        model_3_all_linear1_params = model_3.linear1
        model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
        model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
        model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
        all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                         model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                         model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))

    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
        
##################prediction ##################   

    prediction_1= (model_1(test_x1)[0]>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)[0]>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)[0]>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
################## variable selection ##################   
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_2_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_3_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    
    index=np.nonzero(model_1_weight)[1]
    index=index.tolist()
    index=set(index)
    
    TP=len(trueindex.intersection(index))
    FP=len(index.difference(trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    print(TP)
    print(FP)
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)

tensor(0.2327, dtype=torch.float64)


Trial name,status,loc,lambda1,lambda2,lr,iter,total time (s),my_test_sum_tGM
train_model_74cec_00000,TERMINATED,10.224.0.12:20960,0.05,0.001,0.8,1,15.1887,0.9214
train_model_74cec_00001,TERMINATED,10.224.0.12:20989,0.03,0.003,0.6,1,15.2591,0.861616
train_model_74cec_00002,TERMINATED,10.224.0.12:20991,0.03,0.003,0.6,1,14.7474,0.861616
train_model_74cec_00003,TERMINATED,10.224.0.12:20993,0.05,0.001,0.7,1,14.7375,0.92144
train_model_74cec_00004,TERMINATED,10.224.0.12:20995,0.05,0.003,0.7,1,14.1585,0.918016
train_model_74cec_00005,TERMINATED,10.224.0.12:20997,0.04,0.002,0.7,1,13.9891,0.901301
train_model_74cec_00006,TERMINATED,10.224.0.12:21000,0.04,0.002,0.8,1,13.3633,0.909009
train_model_74cec_00007,TERMINATED,10.224.0.12:21003,0.03,0.002,0.7,1,13.9821,0.876554
train_model_74cec_00008,TERMINATED,10.224.0.12:21005,0.04,0.002,0.7,1,13.9074,0.901301
train_model_74cec_00009,TERMINATED,10.224.0.12:21007,0.04,0.002,0.6,1,14.5867,0.885576


Result for train_model_74cec_00000:
  date: 2022-11-13_21-53-04
  done: false
  experiment_id: aadd056922544ff3bc13cff4fdf03783
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9214001277053729
  node_ip: 10.224.0.12
  pid: 20960
  time_since_restore: 15.188684463500977
  time_this_iter_s: 15.188684463500977
  time_total_s: 15.188684463500977
  timestamp: 1668347584
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 74cec_00000
  warmup_time: 0.0035698413848876953
  
Result for train_model_74cec_00000:
  date: 2022-11-13_21-53-04
  done: true
  experiment_id: aadd056922544ff3bc13cff4fdf03783
  experiment_tag: 0_lambda1=0.0500,lambda2=0.0010,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9214001277053729
  node_ip: 10.224.0.12
  pid: 20960
  time_since_restore: 15.188684463500977
  time_this_iter_s: 15.188684463500977
  time_total_s: 15.188684463500977
  timestamp: 1668347584
  timesteps_since_

Result for train_model_74cec_00010:
  date: 2022-11-13_21-53-06
  done: false
  experiment_id: 0edd17621a274d3bbfcf66fd3fdc488f
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8757745177597664
  node_ip: 10.224.0.12
  pid: 21010
  time_since_restore: 13.954402923583984
  time_this_iter_s: 13.954402923583984
  time_total_s: 13.954402923583984
  timestamp: 1668347586
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 74cec_00010
  warmup_time: 0.0031862258911132812
  
Result for train_model_74cec_00010:
  date: 2022-11-13_21-53-06
  done: true
  experiment_id: 0edd17621a274d3bbfcf66fd3fdc488f
  experiment_tag: 10_lambda1=0.0300,lambda2=0.0020,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8757745177597664
  node_ip: 10.224.0.12
  pid: 21010
  time_since_restore: 13.954402923583984
  time_this_iter_s: 13.954402923583984
  time_total_s: 13.954402923583984
  timestamp: 1668347586
  timesteps_since

Result for train_model_74cec_00002:
  date: 2022-11-13_21-53-07
  done: false
  experiment_id: 7742d6fd1ec94bf58b71f5e210a9c222
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8616155714526897
  node_ip: 10.224.0.12
  pid: 20991
  time_since_restore: 14.747446537017822
  time_this_iter_s: 14.747446537017822
  time_total_s: 14.747446537017822
  timestamp: 1668347587
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 74cec_00002
  warmup_time: 0.009286880493164062
  
Result for train_model_74cec_00002:
  date: 2022-11-13_21-53-07
  done: true
  experiment_id: 7742d6fd1ec94bf58b71f5e210a9c222
  experiment_tag: 2_lambda1=0.0300,lambda2=0.0030,lr=0.6000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8616155714526897
  node_ip: 10.224.0.12
  pid: 20991
  time_since_restore: 14.747446537017822
  time_this_iter_s: 14.747446537017822
  time_total_s: 14.747446537017822
  timestamp: 1668347587
  timesteps_since_r

...

Result for train_model_efc71_00015:
  date: 2022-11-13_21-56-32
  done: false
  experiment_id: ec1260397f494df886709d45938dcda9
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9198999331226148
  node_ip: 10.224.0.12
  pid: 29605
  time_since_restore: 13.160703659057617
  time_this_iter_s: 13.160703659057617
  time_total_s: 13.160703659057617
  timestamp: 1668347792
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: efc71_00015
  warmup_time: 0.004025936126708984
  
Result for train_model_efc71_00015:
  date: 2022-11-13_21-56-32
  done: true
  experiment_id: ec1260397f494df886709d45938dcda9
  experiment_tag: 15_lambda1=0.0400,lambda2=0.0020,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9198999331226148
  node_ip: 10.224.0.12
  pid: 29605
  time_since_restore: 13.160703659057617
  time_this_iter_s: 13.160703659057617
  time_total_s: 13.160703659057617
  timestamp: 1668347792
  timesteps_since_

Result for train_model_efc71_00003:
  date: 2022-11-13_21-56-33
  done: true
  experiment_id: f1fd87d9e8a7472f848838ae62ba0df4
  experiment_tag: 3_lambda1=0.0400,lambda2=0.0030,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9203060714961294
  node_ip: 10.224.0.12
  pid: 29579
  time_since_restore: 14.462144136428833
  time_this_iter_s: 14.462144136428833
  time_total_s: 14.462144136428833
  timestamp: 1668347793
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: efc71_00003
  warmup_time: 0.005913257598876953
  
Result for train_model_efc71_00018:
  date: 2022-11-13_21-56-33
  done: false
  experiment_id: 2dea6f61c6554f0fb5cd0db6e4780d69
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.919706238073226
  node_ip: 10.224.0.12
  pid: 29611
  time_since_restore: 14.607923984527588
  time_this_iter_s: 14.607923984527588
  time_total_s: 14.607923984527588
  timestamp: 1668347793
  timesteps_since_re

Result for train_model_efc71_00027:
  date: 2022-11-13_21-56-33
  done: false
  experiment_id: 3eb49a55c70242889630726c8f7d263f
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9182233816037577
  node_ip: 10.224.0.12
  pid: 29626
  time_since_restore: 14.60342288017273
  time_this_iter_s: 14.60342288017273
  time_total_s: 14.60342288017273
  timestamp: 1668347793
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: efc71_00027
  warmup_time: 0.003470182418823242
  
Result for train_model_efc71_00027:
  date: 2022-11-13_21-56-33
  done: true
  experiment_id: 3eb49a55c70242889630726c8f7d263f
  experiment_tag: 27_lambda1=0.0500,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9182233816037577
  node_ip: 10.224.0.12
  pid: 29626
  time_since_restore: 14.60342288017273
  time_this_iter_s: 14.60342288017273
  time_total_s: 14.60342288017273
  timestamp: 1668347793
  timesteps_since_restor

2022-11-13 21:56:34,588	INFO tune.py:758 -- Total run time: 20.66 seconds (20.45 seconds for the tuning loop).


10
2
9


In [46]:
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)

0.8614072477230371

0.8545941577056129

0.8570089726721278

0.8583333333333334

0.860958499399924

0.8596537268473401

In [49]:
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.99

0.9822222222222223

0.9859740160049977

0.016999999999999998

0.983

In [50]:
vs

array([[10., 10., 10., 10., 10.,  9., 10., 10., 10., 10.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [52]:
vsSPE

[1.0,
 0.9555555555555556,
 0.9777777777777777,
 0.9666666666666667,
 0.9888888888888889,
 1.0,
 0.9888888888888889,
 0.9888888888888889,
 0.9777777777777777,
 0.9777777777777777]